如果想要创建一个全新的实例属性，可以通过一个描述器类的形式来定义它的功能。描述器的定义在8.8中已经进行了描述。其实就是一个包含特定方法的类

#### 描述器协议
`descr.__get__(self, obj, type=None) ----> value`  
`descr.__set__(self, obj, value) ----> None`  
`descr.__delete__(self, obj) ----> None` 
这是所有的描述器方法。一个对象具有其中任意一个方法就会成为描述器，从而在被当做对象属性时重写默认的查找行为  
如果一个对象同时定义了`__get__()` 和 `__set__()`，他叫做资料描述器。  
仅定义了`__get__()`方法的描述器叫做非资料描述器 
  
资料描述器和非资料描述器的区别在于：相对于实例的字典优先级，如果实例字典中有与描述器同名的属性，如果是资料描述器，优先使用资料描述器，如果是非资料描述器，优先使用字典中的属性

如果实例 a 的方法和属性都叫foo时，Python会在访问a.foo时，优先访问实例字典中的属性，因为实例字典是一个非资料描述器

In [4]:
class A:
    def __init__(self,name):
        self.name = name
    
    def name(self):
        print("function",self.name)

a = A("liyang")
a.name

'liyang'

现在把A变成资料描述器

In [10]:
class A:
    def __init__(self,name):
        self.name = name
    
    @property
    def name(self):
        print("function: %s" % self._name)
        return self._name
    
    @name.setter
    def name(self, value):
        print("set name to", value)
        self._name = value

a = A("liyang")
a.name

set name to liyang
function: liyang


'liyang'

如果想要创建一个全新的**实例属性**，可以通过一个描述器类的形式来实现它的功能。例如：

In [11]:
# coding:utf-8
# 增加了整数类型检查的描述器类
class Interger:
    def __init__(self, name):
        self.name = name
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Except an int')
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]


一个描述器就是实现了三个核心的属性访问操作的类。这些方法接收一个实例作为输入，之后相应的去操作**实例底层的字典**  

*** self 表示一个具体的实例本身 ***   
*** cls 表示一个类本身 ***

为了使用一个描述器，需要将这个描述器的实例作为类属性放到一个类中。例如：

In [12]:
class Point:
    # x,y是一个Interger的实例
    x = Interger("x")
    y = Interger("y")
    
    def __init__(self, x, y):
        # 赋值操作时调用实例中的get,set,delete方法
        self.x = x
        self.y = y

In [13]:
p = Point(2, 3)
p.x # calls Point.x.__get__()方法

2

In [16]:
p.y = 2 # calls Point.y.__set__()方法
p.y

2

作为输入，描述器的每一个方法会接收一个操作实例，为了实现请求操作，会响应的操作底层的字典（`__dict__`属性），self.name 属性存储了在实例字典中被实际使用到的key

描述器可以实现大部分Python类的特性，包括 @classmethod 、@staticmethod、@property，甚至是
`__slot__`特性。  
通过定义一个描述器，可以在底层捕获核心的实例操作(get/set/delete)，并且完全可以自定义他们的行为。这是一个强大的工具，他是很多高级库和框架中的重要工具  
描述器一个令人比较困惑的地方是**描述器只能在类级别被定义，而不能为每个实例单独定义**

In [ ]:
# Does NOT work
class Point:
    def __init__(self, x, y):
        self.x = Integer('x') # No! Must be a class variable
        self.y = Integer('y')
        self.x = x
        self.y = y

描述器通常是哪些使用到装饰器或元类的大型框架中的一个组件，同时他们的使用也被隐藏在后面。 
例如：下面是一些更高级的基于描述器的代码，并设计到一个类装饰器:

In [29]:
# 描述器
class Typed:
    def __init__(self, name, excepted_type):
        self.name = name
        self.excepted_type = excepted_type
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.excepted_type):
            raise TypeError('Except :{}'.format(self.excepted_type))
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]

# 装饰器
def typeassert(**kwargs):
    # 传入当前类
    def decorate(cls):
        for name, excepted_type in kwargs.items():
            # 将类型描述符附加到类
            setattr(cls, name, Typed(name, excepted_type))
        return cls
        
    return decorate

# 使用说明
# 指定每个name的类型

@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
stock = Stock("tencent",100,1.0)

最后，如果你只是想简单的自定义某个类的单个属性访问的话就不需要写描述器了。这种情况下property会更加容易。当程序中有很多重复代码是，描述器就会很有用了。